<a href="https://colab.research.google.com/github/taycurran/DS-Unit-2-Linear-Models/blob/master/CURRAN_M3_regression_classification_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Lambda School Data Science

*Unit 2, Sprint 1, Module 3*

---

# Ridge Regression

## Assignment

We're going back to our other **New York City** real estate dataset. Instead of predicting apartment rents, you'll predict property sales prices.

But not just for condos in Tribeca...

- [ ] Use a subset of the data where `BUILDING_CLASS_CATEGORY` == `'01 ONE FAMILY DWELLINGS'` and the sale price was more than 100 thousand and less than 2 million.
- [ ] Do train/test split. Use data from January — March 2019 to train. Use data from April 2019 to test.
- [ ] Do one-hot encoding of categorical features.
- [ ] Do feature selection with `SelectKBest`.
- [ ] Fit a ridge regression model with multiple features. Use the `normalize=True` parameter (or do [feature scaling](https://scikit-learn.org/stable/modules/preprocessing.html) beforehand — use the scaler's `fit_transform` method with the train set, and the scaler's `transform` method with the test set)
- [ ] Get mean absolute error for the test set.
- [ ] As always, commit your notebook to your fork of the GitHub repo.

The [NYC Department of Finance](https://www1.nyc.gov/site/finance/taxes/property-rolling-sales-data.page) has a glossary of property sales terms and NYC Building Class Code Descriptions. The data comes from the [NYC OpenData](https://data.cityofnewyork.us/browse?q=NYC%20calendar%20sales) portal.


## Stretch Goals

Don't worry, you aren't expected to do all these stretch goals! These are just ideas to consider and choose from.

- [ ] Add your own stretch goal(s) !
- [ ] Instead of `Ridge`, try `LinearRegression`. Depending on how many features you select, your errors will probably blow up! 💥
- [ ] Instead of `Ridge`, try [`RidgeCV`](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.RidgeCV.html).
- [ ] Learn more about feature selection:
    - ["Permutation importance"](https://www.kaggle.com/dansbecker/permutation-importance)
    - [scikit-learn's User Guide for Feature Selection](https://scikit-learn.org/stable/modules/feature_selection.html)
    - [mlxtend](http://rasbt.github.io/mlxtend/) library
    - scikit-learn-contrib libraries: [boruta_py](https://github.com/scikit-learn-contrib/boruta_py) & [stability-selection](https://github.com/scikit-learn-contrib/stability-selection)
    - [_Feature Engineering and Selection_](http://www.feat.engineering/) by Kuhn & Johnson.
- [ ] Try [statsmodels](https://www.statsmodels.org/stable/index.html) if you’re interested in more inferential statistical approach to linear regression and feature selection, looking at p values and 95% confidence intervals for the coefficients.
- [ ] Read [_An Introduction to Statistical Learning_](http://faculty.marshall.usc.edu/gareth-james/ISL/ISLR%20Seventh%20Printing.pdf), Chapters 1-3, for more math & theory, but in an accessible, readable way.
- [ ] Try [scikit-learn pipelines](https://scikit-learn.org/stable/modules/compose.html).

In [0]:
%%capture
import sys

# If you're on Colab:
if 'google.colab' in sys.modules:
    DATA_PATH = 'https://raw.githubusercontent.com/LambdaSchool/DS-Unit-2-Applied-Modeling/master/data/'
    !pip install category_encoders==2.*

# If you're working locally:
else:
    DATA_PATH = '../data/'
    
# Ignore this Numpy warning when using Plotly Express:
# FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
import warnings
warnings.filterwarnings(action='ignore', category=FutureWarning, module='numpy')

In [0]:
import pandas as pd
import pandas_profiling

# Read New York City property sales data
df = pd.read_csv(DATA_PATH+'condos/NYC_Citywide_Rolling_Calendar_Sales.csv')

# Change column names: replace spaces with underscores
df.columns = [col.replace(' ', '_') for col in df]

# SALE_PRICE was read as strings.
# Remove symbols, convert to integer
df['SALE_PRICE'] = (
    df['SALE_PRICE']
    .str.replace('$','')
    .str.replace('-','')
    .str.replace(',','')
    .astype(int)
)

In [0]:
# BOROUGH is a numeric column, but arguably should be a categorical feature,
# so convert it from a number to a string
df['BOROUGH'] = df['BOROUGH'].astype(str)

In [0]:
# Reduce cardinality for NEIGHBORHOOD feature

# Get a list of the top 10 neighborhoods
top10 = df['NEIGHBORHOOD'].value_counts()[:10].index

# At locations where the neighborhood is NOT in the top 10, 
# replace the neighborhood with 'OTHER'
df.loc[~df['NEIGHBORHOOD'].isin(top10), 'NEIGHBORHOOD'] = 'OTHER'

## MY CODE STARTS HERE

In [103]:
df.head(2)

,BOROUGH,NEIGHBORHOOD,BUILDING_CLASS_CATEGORY,TAX_CLASS_AT_PRESENT,BLOCK,LOT,EASE-MENT,BUILDING_CLASS_AT_PRESENT,ADDRESS,APARTMENT_NUMBER,ZIP_CODE,RESIDENTIAL_UNITS,COMMERCIAL_UNITS,TOTAL_UNITS,LAND_SQUARE_FEET,GROSS_SQUARE_FEET,YEAR_BUILT,TAX_CLASS_AT_TIME_OF_SALE,BUILDING_CLASS_AT_TIME_OF_SALE,SALE_PRICE,SALE_DATE
0,1,OTHER,13 CONDOS - ELEVATOR APARTMENTS,2,716,1246,NaN,R4,"447 WEST 18TH STREET, PH12A",PH12A,10011.0,1.0,0.0,1.0,"10,733",1979.0,2007.0,2,R4,0,01/01/2019
1,1,OTHER,21 OFFICE BUILDINGS,4,812,68,NaN,O5,144 WEST 37TH STREET,NaN,10018.0,0.0,6.0,6.0,"2,962",15435.0,1920.0,4,O5,0,01/01/2019


In [104]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23040 entries, 0 to 23039
Data columns (total 21 columns):
BOROUGH                           23040 non-null object
NEIGHBORHOOD                      23040 non-null object
BUILDING_CLASS_CATEGORY           23040 non-null object
TAX_CLASS_AT_PRESENT              23039 non-null object
BLOCK                             23040 non-null int64
LOT                               23040 non-null int64
EASE-MENT                         0 non-null float64
BUILDING_CLASS_AT_PRESENT         23039 non-null object
ADDRESS                           23040 non-null object
APARTMENT_NUMBER                  5201 non-null object
ZIP_CODE                          23039 non-null float64
RESIDENTIAL_UNITS                 23039 non-null float64
COMMERCIAL_UNITS                  23039 non-null float64
TOTAL_UNITS                       23039 non-null float64
LAND_SQUARE_FEET                  22987 non-null object
GROSS_SQUARE_FEET                 23039 non-null floa

### **Subset of Data**

In [105]:
# Describe Condition
condBuildClass = df['BUILDING_CLASS_CATEGORY'] == '01 ONE FAMILY DWELLINGS'
condSalePrice = (100000 < df['SALE_PRICE']) & (df['SALE_PRICE'] < 2000000)
condSubSet = (condBuildClass & condSalePrice)
subSet = df[condSubSet]

print(subSet.shape)
subSet.head(2)

(3151, 21)


,BOROUGH,NEIGHBORHOOD,BUILDING_CLASS_CATEGORY,TAX_CLASS_AT_PRESENT,BLOCK,LOT,EASE-MENT,BUILDING_CLASS_AT_PRESENT,ADDRESS,APARTMENT_NUMBER,ZIP_CODE,RESIDENTIAL_UNITS,COMMERCIAL_UNITS,TOTAL_UNITS,LAND_SQUARE_FEET,GROSS_SQUARE_FEET,YEAR_BUILT,TAX_CLASS_AT_TIME_OF_SALE,BUILDING_CLASS_AT_TIME_OF_SALE,SALE_PRICE,SALE_DATE
44,3,OTHER,01 ONE FAMILY DWELLINGS,1,5495,801,NaN,A9,4832 BAY PARKWAY,NaN,11230.0,1.0,0.0,1.0,"6,800",1325.0,1930.0,1,A9,550000,01/01/2019
61,4,OTHER,01 ONE FAMILY DWELLINGS,1,7918,72,NaN,A1,80-23 232ND STREET,NaN,11427.0,1.0,0.0,1.0,"4,000",2001.0,1940.0,1,A1,200000,01/01/2019


### **Train/Test Split**

In [106]:
# Separate Train and Test Data According to Date
subSet['SALE_DATE'] = pd.to_datetime(subSet['SALE_DATE'], 
                                       infer_datetime_format=True)
cutOffDate = pd.to_datetime('2019-04-01')
train = subSet[subSet['SALE_DATE'] < cutOffDate]
test = subSet[subSet['SALE_DATE'] >= cutOffDate]

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [107]:
print(train.shape)
print(test.shape)

(2507, 21)
(644, 21)


### **One-Hot Encoding of Categorical Features**

In [0]:
# Establish Target Variable
target = 'SALE_PRICE'

Some Exploration Before One-Hot Encoding...

In [109]:
# Exploration of Categorical Data

train.describe(exclude='number').T.sort_values(by='unique')

,count,unique,top,freq,first,last
BUILDING_CLASS_CATEGORY,2507,1,01 ONE FAMILY DWELLINGS,2507,NaT,NaT
APARTMENT_NUMBER,1,1,RP.,1,NaT,NaT
TAX_CLASS_AT_PRESENT,2507,2,1,2476,NaT,NaT
BOROUGH,2507,5,4,1204,NaT,NaT
NEIGHBORHOOD,2507,6,OTHER,2382,NaT,NaT
BUILDING_CLASS_AT_TIME_OF_SALE,2507,11,A1,919,NaT,NaT
BUILDING_CLASS_AT_PRESENT,2507,13,A1,919,NaT,NaT
SALE_DATE,2507,68,2019-01-31 00:00:00,78,2019-01-01,2019-03-30
LAND_SQUARE_FEET,2507,887,"4,000",234,NaT,NaT
ADDRESS,2507,2497,104-18 187TH STREET,2,NaT,NaT


In [110]:
# Does BOROUGH of Apt. Make a Difference?
train.groupby('BOROUGH')[target].mean()

BOROUGH
1    739000.000000
2    473104.623967
3    768734.633166
4    649005.172757
5    537305.802115
Name: SALE_PRICE, dtype: float64

In [111]:
# Does NEIGHBORHOOD of Apt. Make a Difference?
train.groupby('NEIGHBORHOOD')[target].mean()

NEIGHBORHOOD
ASTORIA               1.001955e+06
BEDFORD STUYVESANT    6.215972e+05
BOROUGH PARK          1.008917e+06
FLUSHING-NORTH        8.689417e+05
FOREST HILLS          1.210753e+06
OTHER                 6.056645e+05
Name: SALE_PRICE, dtype: float64

In [112]:
# Does BUILDING_CLASS_AT_TIME_OF_SALE of Apt. Make a Difference?
train.groupby('BUILDING_CLASS_AT_TIME_OF_SALE')[target].mean()

BUILDING_CLASS_AT_TIME_OF_SALE
A0    6.884526e+05
A1    6.410574e+05
A2    5.991551e+05
A3    1.199412e+06
A4    9.254561e+05
A5    5.718045e+05
A6    2.344273e+05
A8    3.929000e+05
A9    6.311994e+05
S0    1.925000e+06
S1    8.544496e+05
Name: SALE_PRICE, dtype: float64

In [113]:
# Does BUILDING_CLASS_AT_PRESENT of Apt. Make a Difference?
train.groupby('BUILDING_CLASS_AT_PRESENT')[target].mean()

BUILDING_CLASS_AT_PRESENT
A0    6.884526e+05
A1    6.410574e+05
A2    5.980673e+05
A3    1.199412e+06
A4    9.254561e+05
A5    5.716279e+05
A6    2.344273e+05
A8    3.929000e+05
A9    6.311994e+05
B2    8.300000e+05
B3    9.530000e+05
S0    1.925000e+06
S1    8.544496e+05
Name: SALE_PRICE, dtype: float64

One-Hot Encoding Process

In [114]:
# Separate Out Features with High Cardinality from One Hot Encoding Process
# Also Remove Target Vector and Column that for Our Subset has All NaNs
highCardinality = ['SALE_DATE', 'LAND_SQUARE_FEET', 'ADDRESS']
allNaNs = ['EASE-MENT']
features = train.columns.drop([target] + highCardinality + allNaNs)
features

Index(['BOROUGH', 'NEIGHBORHOOD', 'BUILDING_CLASS_CATEGORY',
       'TAX_CLASS_AT_PRESENT', 'BLOCK', 'LOT', 'BUILDING_CLASS_AT_PRESENT',
       'APARTMENT_NUMBER', 'ZIP_CODE', 'RESIDENTIAL_UNITS', 'COMMERCIAL_UNITS',
       'TOTAL_UNITS', 'GROSS_SQUARE_FEET', 'YEAR_BUILT',
       'TAX_CLASS_AT_TIME_OF_SALE', 'BUILDING_CLASS_AT_TIME_OF_SALE'],
      dtype='object')

In [0]:
# Set Up Features Matrices and Target Vectors
X_train = train[features]
y_train = train[target]
X_test = test[features]
y_test = test[target]

In [116]:
# X_train Before One-Hot Encoding
X_train.head(2).T

,44,61
BOROUGH,3,4
NEIGHBORHOOD,OTHER,OTHER
BUILDING_CLASS_CATEGORY,01 ONE FAMILY DWELLINGS,01 ONE FAMILY DWELLINGS
TAX_CLASS_AT_PRESENT,1,1
BLOCK,5495,7918
LOT,801,72
BUILDING_CLASS_AT_PRESENT,A9,A1
APARTMENT_NUMBER,NaN,NaN
ZIP_CODE,11230,11427
RESIDENTIAL_UNITS,1,1


- Use the **`fit_transform`** method on the **train** set
- Use the **`transform`** method on **validation / test** sets

In [0]:
# Import the Category_Encoders Library
import category_encoders as ce

# Instantiate the One-Hot Encoder Object (or Tool)
encoder = ce.OneHotEncoder(use_cat_names=True)

# fit_transform the encoder tool to X_train Dataframe
X_train = encoder.fit_transform(X_train)

In [0]:
# transform the encoder tool to the X_test Dataframe Because...
# both train_X and test_X must have the SAME Columns in the SAME Order.
X_test = encoder.transform(X_test)

In [119]:
# X_train After One-Hot Encoding
print(X_train.shape)
X_train.head(2).T

(2507, 49)


,44,61
BOROUGH_3,1.0,0.0
BOROUGH_4,0.0,1.0
BOROUGH_2,0.0,0.0
BOROUGH_5,0.0,0.0
BOROUGH_1,0.0,0.0
NEIGHBORHOOD_OTHER,1.0,1.0
NEIGHBORHOOD_FLUSHING-NORTH,0.0,0.0
NEIGHBORHOOD_BEDFORD STUYVESANT,0.0,0.0
NEIGHBORHOOD_FOREST HILLS,0.0,0.0
NEIGHBORHOOD_BOROUGH PARK,0.0,0.0


In [120]:
print(X_train.shape)
X_train.describe()

(2507, 49)


,BOROUGH_3,BOROUGH_4,BOROUGH_2,BOROUGH_5,BOROUGH_1,NEIGHBORHOOD_OTHER,NEIGHBORHOOD_FLUSHING-NORTH,NEIGHBORHOOD_BEDFORD STUYVESANT,NEIGHBORHOOD_FOREST HILLS,NEIGHBORHOOD_BOROUGH PARK,NEIGHBORHOOD_ASTORIA,BUILDING_CLASS_CATEGORY_01 ONE FAMILY DWELLINGS,TAX_CLASS_AT_PRESENT_1,TAX_CLASS_AT_PRESENT_1D,BLOCK,LOT,BUILDING_CLASS_AT_PRESENT_A9,BUILDING_CLASS_AT_PRESENT_A1,BUILDING_CLASS_AT_PRESENT_A5,BUILDING_CLASS_AT_PRESENT_A0,BUILDING_CLASS_AT_PRESENT_A2,BUILDING_CLASS_AT_PRESENT_A3,BUILDING_CLASS_AT_PRESENT_S1,BUILDING_CLASS_AT_PRESENT_A4,BUILDING_CLASS_AT_PRESENT_A6,BUILDING_CLASS_AT_PRESENT_A8,BUILDING_CLASS_AT_PRESENT_B2,BUILDING_CLASS_AT_PRESENT_S0,BUILDING_CLASS_AT_PRESENT_B3,APARTMENT_NUMBER_nan,APARTMENT_NUMBER_RP.,ZIP_CODE,RESIDENTIAL_UNITS,COMMERCIAL_UNITS,TOTAL_UNITS,GROSS_SQUARE_FEET,YEAR_BUILT,TAX_CLASS_AT_TIME_OF_SALE,BUILDING_CLASS_AT_TIME_OF_SALE_A9,BUILDING_CLASS_AT_TIME_OF_SALE_A1,BUILDING_CLASS_AT_TIME_OF_SALE_A5,BUILDING_CLASS_AT_TIME_OF_SALE_A0,BUILDING_CLASS_AT_TIME_OF_SALE_A2,BUILDING_CLASS_AT_TIME_OF_SALE_A3,BUILDING_CLASS_AT_TIME_OF_SALE_S1,BUILDING_CLASS_AT_TIME_OF_SALE_A4,BUILDING_CLASS_AT_TIME_OF_SALE_A6,BUILDING_CLASS_AT_TIME_OF_SALE_A8,BUILDING_CLASS_AT_TIME_OF_SALE_S0
count,2507.000000,2507.000000,2507.000000,2507.000000,2507.000000,2507.00000,2507.000000,2507.000000,2507.000000,2507.000000,2507.000000,2507.0,2507.000000,2507.000000,2507.000000,2507.000000,2507.000000,2507.000000,2507.000000,2507.000000,2507.000000,2507.000000,2507.000000,2507.000000,2507.000000,2507.000000,2507.000000,2507.000000,2507.000000,2507.000000,2507.000000,2507.000000,2507.000000,2507.000000,2507.000000,2507.000000,2507.000000,2507.0,2507.000000,2507.000000,2507.000000,2507.000000,2507.000000,2507.000000,2507.000000,2507.000000,2507.000000,2507.000000,2507.000000
mean,0.158755,0.480255,0.096530,0.264061,0.000399,0.95014,0.030714,0.003191,0.006781,0.004787,0.004388,1.0,0.987635,0.012365,6758.303949,75.778221,0.076984,0.366574,0.310730,0.026725,0.163941,0.015158,0.015556,0.005185,0.005584,0.012365,0.000399,0.000399,0.000399,0.999601,0.000399,10993.398484,0.987635,0.016354,1.003989,1473.744715,1944.766653,1.0,0.076984,0.366574,0.310730,0.026725,0.164739,0.015158,0.015556,0.005185,0.005584,0.012365,0.000399
std,0.365521,0.499710,0.295375,0.440920,0.019972,0.21770,0.172576,0.056411,0.082084,0.069033,0.066108,0.0,0.110532,0.110532,3975.909029,157.531138,0.266620,0.481965,0.462885,0.161311,0.370296,0.122204,0.123776,0.071838,0.074535,0.110532,0.019972,0.019972,0.019972,0.019972,0.019972,494.291462,0.110532,0.129966,0.171794,599.217635,27.059337,0.0,0.266620,0.481965,0.462885,0.161311,0.371019,0.122204,0.123776,0.071838,0.074535,0.110532,0.019972
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,1.0,0.000000,0.000000,21.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,10301.000000,0.000000,0.000000,0.000000,0.000000,1890.000000,1.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,1.00000,0.000000,0.000000,0.000000,0.000000,0.000000,1.0,1.000000,0.000000,3837.500000,21.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,10314.000000,1.000000,0.000000,1.000000,1144.000000,1925.000000,1.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,1.00000,0.000000,0.000000,0.000000,0.000000,0.000000,1.0,1.000000,0.000000,6022.000000,42.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,11234.000000,1.000000,0.000000,1.000000,1368.000000,1940.000000,1.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0

### **Feature Selection with `SelectKBest`**

In [140]:
# import SelectKBest Class and f_regression from SciKitLearn Feature_Selection
from sklearn.feature_selection import SelectKBest, f_regression

# Instantiate SelectKBest Tool Using f_regression for Scoring
# Keep top K Scores of f_regression
selector = SelectKBest(score_func=f_regression, k=20)

# Fit the SelectKBest Selector Tool to the Training Data
X_trainSelected = selector.fit_transform(X_train, y_train)

/usr/local/lib/python3.6/dist-packages/sklearn/feature_selection/_univariate_selection.py:299: RuntimeWarning: divide by zero encountered in true_divide
  corr /= X_norms
/usr/local/lib/python3.6/dist-packages/sklearn/feature_selection/_univariate_selection.py:304: RuntimeWarning: invalid value encountered in true_divide
  F = corr ** 2 / (1 - corr ** 2) * degrees_of_freedom
/usr/local/lib/python3.6/dist-packages/scipy/stats/_distn_infrastructure.py:903: RuntimeWarning: invalid value encountered in greater
  return (a < x) & (x < b)
/usr/local/lib/python3.6/dist-packages/scipy/stats/_distn_infrastructure.py:903: RuntimeWarning: invalid value encountered in less
  return (a < x) & (x < b)
/usr/local/lib/python3.6/dist-packages/scipy/stats/_distn_infrastructure.py:1912: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= _a)


In [0]:
# Transform X_train to Match Changes Made from Selector Tool
X_testSelected = selector.transform(X_test)

In [142]:
# Which features were selected?
# Selected Columns is a Boolian Array Showing Which Columns Were Selected by...
# SelectKBest
selectedMask = selector.get_support()
len(selectedMask) == len(X_train.columns)

True

In [143]:
# Columns Selected by the SelectKBest Tool
selectedNames = X_train.columns[selectedMask]
unselectedNames = X_train.columns[~selectedMask]

print('Features Selected:\n')
for name in selectedNames:
  print(name)

Features Selected:

BOROUGH_3
BOROUGH_2
BOROUGH_5
NEIGHBORHOOD_OTHER
NEIGHBORHOOD_FLUSHING-NORTH
NEIGHBORHOOD_FOREST HILLS
NEIGHBORHOOD_BOROUGH PARK
BLOCK
BUILDING_CLASS_AT_PRESENT_A5
BUILDING_CLASS_AT_PRESENT_A3
BUILDING_CLASS_AT_PRESENT_S1
BUILDING_CLASS_AT_PRESENT_A6
ZIP_CODE
COMMERCIAL_UNITS
TOTAL_UNITS
GROSS_SQUARE_FEET
BUILDING_CLASS_AT_TIME_OF_SALE_A5
BUILDING_CLASS_AT_TIME_OF_SALE_A3
BUILDING_CLASS_AT_TIME_OF_SALE_S1
BUILDING_CLASS_AT_TIME_OF_SALE_A6


In [0]:
# Use a For Loop to Determine How Many Features Should Be Selected

# Import Necessary Libraries
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error

# Establish Range
rangeOneFeaturetoAllFeatures = range(1, len(X_train.columns)+1)

for k in rangeOneFeaturetoAllFeatures:
  print(f'{k} features')

  selector = SelectKBest(score_func=f_regression, k=k)
  X_trainSelected = selector.fit_transform(X_train, y_train)
  X_testSelected = selector.transform(X_test)

  model = LinearRegression()
  model.fit(X_trainSelected, y_train)
  y_pred = model.predict(X_testSelected)

  mae = mean_absolute_error(y_test, y_pred)

  print(f"Test Mean Absolute Error: {mae} \n")

# There are minimal changes from k=20 to k=38
# Though k=38 Generates the Smalles Mean Absolute Error

In [135]:
#for feature, coef in zip(X_train.columns, model.coef_):
  #print(f"{feature}: {coef}")
dfCoef = pd.DataFrame({'FeatureName':X_train.columns, 
                       'Coefficient': model.coef_})
dfCoef.head()

,FeatureName,Coefficient
0,BOROUGH_3,9.436469e+13
1,BOROUGH_4,9.436469e+13
2,BOROUGH_2,9.436469e+13
3,BOROUGH_5,9.436469e+13
4,BOROUGH_1,9.436469e+13


### **Ridge Regression Model**

In [0]:
import matplotlib.pyplot as plt
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_absolute_error

def ridge_apartments(alpha):
    """
    Fit & plot a ridge regression model.

    alpha : positive float, regularization strength
    """

    # Load data
    apartments_X = X_train[selectedNames]
    apartments_y = y_train
    
    # Fit linear model
    ridge = Ridge(alpha=alpha, normalize=True)
    ridge.fit(apartments_X, apartments_y)

    # Get predictions
    apartPred = ridge.predict(X_test[selectedNames])

    # Get MAE
    apartMAE = mean_absolute_error(y_test, apartPred)

    print(f"MAE: {apartMAE}")

### **Mean Absolute Error for Test Set**

In [161]:
ridge_apartments(0.1)

MAE: 160009.0387295712
